In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [3]:
def basic_transform():
    def apply(image, mask):
        return image, mask
    return apply

In [4]:
class ImageMaskDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.images = [f for f in os.listdir(img_dir) if f.endswith(('.jpg','.png','.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        filename = self.images[idx]
        path = os.path.join(self.img_dir, filename)

        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

        if self.transform:
            image, mask = self.transform()(image, mask)

        image = cv2.resize(image, (256, 256))
        mask  = cv2.resize(mask,  (256, 256))

        img_tensor = torch.from_numpy(image.astype(np.float32) / 255.).permute(2,0,1)
        mask_tensor = torch.from_numpy(mask).long()

        return img_tensor, mask_tensor, filename

In [5]:
def save_output(image, mask, name, out_dir):
    img_np = (image.permute(1,2,0).numpy() * 255).astype(np.uint8)
    mask_np = mask.numpy().astype(np.uint8)

    cv2.imwrite(os.path.join(out_dir, f"processed_{name}"), img_np)
    cv2.imwrite(os.path.join(out_dir, f"mask_{name}"), mask_np)

In [6]:
def run(img_dir, out_dir):
    dataset = ImageMaskDataset(img_dir, transform=basic_transform)
    loader = DataLoader(dataset, batch_size=1, shuffle=False)

    os.makedirs(out_dir, exist_ok=True)

    for img, mask, name in loader:
        save_output(img[0], mask[0], name[0], out_dir)

    print("✔ Processing Completed Successfully")

In [7]:
image_dir = "/content/drive/MyDrive/AI_Vision_Extract_Nov25/data/COCO2017_SAMPLE"
out_dir   = "/content/processed_output"

run(image_dir, out_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/AI_Vision_Extract_Nov25/data/COCO2017_SAMPLE'